<a href="https://colab.research.google.com/github/LalithShankar/sentiment-analysis/blob/master/Bert_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install cdqa


In [1]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.converters import pdf_converter
from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline
from cdqa.utils.download import download_model

/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
download_model(model='bert-squad_1.1', dir='./models')

In [5]:
def download_pdf():
    import os
    import wget
    directory = './data/pdf/'
    models_url = [
      'https://invest.bnpparibas.com/documents/1q19-pr-12648',
      'https://invest.bnpparibas.com/documents/4q18-pr-18000',
      'https://invest.bnpparibas.com/documents/4q17-pr'
    ]

    print('\nDownloading PDF files...')

    if not os.path.exists(directory):
        os.makedirs(directory)
    for url in models_url:
        wget.download(url=url, out=directory)

download_pdf()

In [6]:
df = pdf_converter(directory_path='./data/pdf/')
df.head()

2020-02-03 07:22:09,400 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar to /tmp/tika-server.jar.
2020-02-03 07:22:18,115 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar.md5 to /tmp/tika-server.jar.md5.
2020-02-03 07:22:19,826 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


,title,paragraphs
0,4q18-pr2 (1),"[2018 FULL YEAR RESULTS PRESS RELEASE Paris,..."
1,1q19-pr-12648 (1),[FIRST QUARTER 2019 RESULTS PRESS RELEASE Pa...
2,4q17-pr,"[2017 FULL YEAR RESULTS PRESS RELEASE Paris,..."
3,1q19-pr-12648,[FIRST QUARTER 2019 RESULTS PRESS RELEASE Pa...
4,4q17-pr (1),"[2017 FULL YEAR RESULTS PRESS RELEASE Paris,..."


In [7]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib', max_df=1.0)

# Fit Retriever to documents
cdqa_pipeline.fit_retriever(df=df)

100%|██████████| 231508/231508 [00:00<00:00, 417899.84B/s]


QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po..._size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=1.0, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [0]:
query = 'How many contracts did BNP Paribas Cardif sell in 2019?'
prediction = cdqa_pipeline.predict(query)

In [10]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: How many contracts did BNP Paribas Cardif sell in 2019?
answer: 140,000
title: 1q19-pr-12648 (1)
paragraph: 3 Excluding PEL/CEL effects of +2 million euros compared to +1 million euros in the first quarter 2018    5 RESULTS AS AT 31 MARCH 2019   The new property and casualty offering launched in May 2018 as part of the partnership between  BNP Paribas Cardif and Matmut (Cardif IARD) recorded good growth with already  140,000 contracts sold as at 31 March 2019.   The business is accelerating individual customers’ mobile uses and developing self-care features with the roll-out of the conversational chatbots Telmi in the Mes comptes BNP Paribas app and Helloïz at Hello bank!.  
